### Fill In Missing 

In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from pyspark.sql.functions import pandas_udf, PandasUDFType
import pathlib
import time
import pyspark.pandas as ps

import numpy as np

from datetime import date, datetime, timedelta
from pyspark.pandas.indexes.datetimes import DatetimeIndex
import pyspark
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import col,isnan,when,count, when, col, rank, lit, monotonically_increasing_id, date_trunc, udf, min, max, explode
from pyspark.sql.types import StructType, StructField, TimestampType, IntegerType, FloatType, DateType, StringType

In [2]:
conf = pyspark.SparkConf().setAll([\
    ('spark.app.name', 'ReduceData')])
spark = SparkSession.builder.config(conf=conf)\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/16 20:26:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
glucose_data_schema=StructType([StructField('NumId', IntegerType(), True),
                                        StructField('PatientId', StringType(), True),
                                        StructField('Value', FloatType(), True),
                                        StructField('GlucoseDisplayTime', TimestampType(), True),
                                        StructField('GlucoseDisplayTimeRaw', StringType(), True),
                                        StructField('GlucoseDisplayDate', DateType(), True)])

In [7]:
data_types = ['train', 'test', 'val']

for data_type in data_types: 
    data_location = "/cephfs/train_test_val/" + data_type
    allPaths = [str(x) for x in list(pathlib.Path(data_location).glob('*.parquet')) if 'part-00' in str(x)]
    path_counter = 0
    
    for path in allPaths:
        gluc = pd.read_parquet(path, columns=['NumId','GlucoseDisplayTime', 'Value'])
        gluc['GlucoseDisplayTime'] = gluc['GlucoseDisplayTime'].dt.floor('Min')
        gluc = gluc.sort_values(by=['NumId', 'GlucoseDisplayTime'])

        min_max = gluc.groupby('NumId')\
                        .agg({'GlucoseDisplayTime' : ['min','max']})

        merge_df = pd.DataFrame(columns=['GlucoseDisplayTime', 'NumId'])
        starttime = time.time()
        last_idx = len(min_max)-1

        index_counter = 0
        for idx, row in min_max.iterrows():
            #grab all potential dates in range

            min_val = row['GlucoseDisplayTime']['min']
            max_val = row['GlucoseDisplayTime']['max']

            date_df = pd.DataFrame(pd.date_range(min_val, max_val, freq='5min'),\
                                   columns=['GlucoseDisplayTime'])  

            # merge dates with big pypsark df
            id_df = gluc[gluc['NumId'] == idx]

            mean = id_df.Value.mean().round()

            id_df.set_index('GlucoseDisplayTime', inplace=True)    

            date_df.set_index('GlucoseDisplayTime', inplace=True)

            merged = id_df.join(date_df, how='outer',\
                                on='GlucoseDisplayTime', sort=True)

            merged['IsFilledIn'] = 0
            merged.loc[merged.Value.isna(), 'IsFilledIn'] = 1        
            merged.loc[merged.Value.isna(), 'Value'] = mean

            merged['NumId'] = idx

            merged.reset_index(inplace=True)

            merged = merged.drop(columns=['index'])

            merged['TimeLag'] = np.concatenate((merged['GlucoseDisplayTime'].iloc[0],\
                                                np.array(merged['GlucoseDisplayTime'].iloc[:-1].values)), axis=None)\
                                .astype('datetime64[ns]')

            merged['Diff'] = (merged['TimeLag'] - merged['GlucoseDisplayTime']).dt.seconds

            len_merged = len(merged)

            # get all index of rows with diff less than 5 mins, add 1 to remove next row, 
            # dont include last row to delete
            indexes_to_remove = [x for x in merged[merged['Diff'] < 300].index + 1 if x < len_merged & x != 0]

            if len(indexes_to_remove) > 0:
                merged = merged.drop(indexes_to_remove)

            # its ready freddy for some interpoletty
            # merged DF is the dataframe ready to go into interpolation function

            # fill with mean

            merged = merged.drop(columns=['TimeLag', 'Diff'])

            if ((index_counter % 25 != 0) and index_counter != last_idx) or (index_counter == 0):
                merge_df = pd.concat([merge_df, merged])
            elif (index_counter % 25 == 0) or (index_counter == last_idx):
                merge_df = merge_df.astype({'GlucoseDisplayTime': 'datetime64[ns]'})
                
                merge_df.to_parquet('/cephfs/interpolation/' + data_type + '/parquet_' + str(path_counter) + '_' + str(index_counter) + '.parquet')
                merge_df = pd.DataFrame(columns=['GlucoseDisplayTime', 'NumId'])
               
            
            index_counter += 1

        path_counter += 1
    
    print(data_type + ' done')
        

train
   GlucoseDisplayTime NumId  Value  IsFilledIn
0 2022-02-01 00:01:00    76  153.0         0.0
1 2022-02-01 00:06:00    76  149.0         0.0
2 2022-02-01 00:11:00    76  146.0         0.0
3 2022-02-01 00:16:00    76  174.0         1.0
4 2022-02-01 00:21:00    76  140.0         0.0
test
   GlucoseDisplayTime NumId  Value  IsFilledIn
0 2022-02-01 00:14:00    69  114.0         0.0
1 2022-02-01 00:19:00    69  143.0         1.0
2 2022-02-01 00:24:00    69  143.0         1.0
3 2022-02-01 00:29:00    69  113.0         0.0
4 2022-02-01 00:34:00    69  143.0         1.0
val
   GlucoseDisplayTime NumId  Value  IsFilledIn
0 2022-02-16 17:49:00    97  203.0         0.0
1 2022-02-16 17:54:00    97  201.0         1.0
2 2022-02-16 17:59:00    97  201.0         1.0
3 2022-02-16 18:04:00    97  201.0         0.0
4 2022-02-16 18:09:00    97  201.0         1.0


testing if it worked

In [5]:
train_sample = spark.read.format('parquet').load('/cephfs/interpolation/train/parquet_0_25.parquet')

In [13]:
train_sample.sort('NumId', 'GlucoseDisplayTime').show(20)

+-------------------+-----+---------+------------------+----------+-----------------+
| GlucoseDisplayTime|NumId|    Value|GlucoseDisplayDate|IsFilledIn|__index_level_0__|
+-------------------+-----+---------+------------------+----------+-----------------+
|2022-02-01 00:01:00|   76|    153.0|        2022-02-01|       0.0|                0|
|2022-02-01 00:06:00|   76|    149.0|        2022-02-01|       0.0|                1|
|2022-02-01 00:11:00|   76|    146.0|        2022-02-01|       0.0|                2|
|2022-02-01 00:16:00|   76|174.41835|        2022-02-01|       1.0|                3|
|2022-02-01 00:21:00|   76|    140.0|        2022-02-01|       0.0|                4|
|2022-02-01 00:26:00|   76|    137.0|        2022-02-01|       0.0|                5|
|2022-02-01 00:31:00|   76|    135.0|        2022-02-01|       0.0|                6|
|2022-02-01 00:36:00|   76|    133.0|        2022-02-01|       0.0|                7|
|2022-02-01 00:41:00|   76|174.41835|        2022-02-0

In [9]:
test_sample = spark.read.format('parquet').load('/cephfs/interpolation/test/parquet_0_25.parquet')

In [12]:
test_sample.sort('NumId', 'GlucoseDisplayTime').show(20)

+-------------------+-----+---------+------------------+----------+-----------------+
| GlucoseDisplayTime|NumId|    Value|GlucoseDisplayDate|IsFilledIn|__index_level_0__|
+-------------------+-----+---------+------------------+----------+-----------------+
|2022-02-01 00:14:00|   69|    114.0|        2022-02-01|       0.0|                0|
|2022-02-01 00:19:00|   69|142.54594|        2022-02-01|       1.0|                1|
|2022-02-01 00:24:00|   69|142.54594|        2022-02-01|       1.0|                2|
|2022-02-01 00:29:00|   69|    113.0|        2022-02-01|       0.0|                3|
|2022-02-01 00:34:00|   69|142.54594|        2022-02-01|       1.0|                4|
|2022-02-01 00:39:00|   69|142.54594|        2022-02-01|       1.0|                5|
|2022-02-01 00:44:00|   69|142.54594|        2022-02-01|       1.0|                6|
|2022-02-01 00:49:00|   69|142.54594|        2022-02-01|       1.0|                7|
|2022-02-01 00:54:00|   69|142.54594|        2022-02-0

In [15]:
val_sample = spark.read.format('parquet').load('/cephfs/train_test_val/val/parquet_0_25.parquet')

In [16]:
val_sample.sort('NumId', 'GlucoseDisplayTime').show(20)

+-----+--------------------+-----+-------------------+---------------------+------------------+
|NumId|           PatientId|Value| GlucoseDisplayTime|GlucoseDisplayTimeRaw|GlucoseDisplayDate|
+-----+--------------------+-----+-------------------+---------------------+------------------+
|  134|H24YXOcfrUrg312CB...|122.0|2022-02-01 07:42:00| 2022-02-01T07:42:...|        2022-02-01|
|  134|H24YXOcfrUrg312CB...|186.0|2022-02-01 10:27:00| 2022-02-01T10:27:...|        2022-02-01|
|  134|H24YXOcfrUrg312CB...|138.0|2022-02-01 08:47:00| 2022-02-01T08:47:...|        2022-02-01|
|  134|H24YXOcfrUrg312CB...|135.0|2022-02-01 19:02:00| 2022-02-01T19:02:...|        2022-02-01|
|  134|H24YXOcfrUrg312CB...|108.0|2022-02-01 13:22:00| 2022-02-01T13:22:...|        2022-02-01|
|  134|H24YXOcfrUrg312CB...|159.0|2022-02-01 01:12:00| 2022-02-01T01:12:...|        2022-02-01|
|  134|H24YXOcfrUrg312CB...|100.0|2022-02-01 17:12:00| 2022-02-01T17:12:...|        2022-02-01|
|  134|H24YXOcfrUrg312CB...|169.0|2022-0